<a href="https://colab.research.google.com/github/p25-c4/plate_project/blob/main/%EB%B9%88%EB%8F%84%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 목표변수와 범주형 설명변수 관계 확인

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df_raw = pd.read_csv('/home/piai/BigData/Plate_project/SCALE불량.csv', encoding='euc-kr')
df_raw.head()

,plate_no,rolling_date,scale,spec_long,spec_country,steel_kind,pt_thick,pt_width,pt_length,hsb,...,fur_heat_temp,fur_heat_time,fur_soak_temp,fur_soak_time,fur_total_time,fur_ex_temp,rolling_method,rolling_temp,descaling_count,work_group
0,PLT_1001,03JAN2023:07:07:53,양품,AB/EH32-TM,미국,T,32,3700,15100,적용,...,1144,116,1133,59,259,1133,TMCP(온도제어),934,8,1조
1,PLT_1002,03JAN2023:07:21:22,양품,AB/EH32-TM,미국,T,32,3700,15100,적용,...,1144,122,1135,53,238,1135,TMCP(온도제어),937,8,1조
2,PLT_1003,03JAN2023:07:31:15,양품,NV-E36-TM,영국,T,33,3600,19200,적용,...,1129,116,1121,55,258,1121,TMCP(온도제어),889,8,1조
3,PLT_1004,03JAN2023:07:41:01,양품,NV-E36-TM,영국,T,33,3600,19200,적용,...,1152,125,1127,68,266,1127,TMCP(온도제어),885,8,1조
4,PLT_1005,03JAN2023:07:52:40,양품,BV-EH36-TM,프랑스,T,38,3100,13300,적용,...,1140,134,1128,48,246,1128,TMCP(온도제어),873,8,1조


In [ ]:
def fun_print_crosstab(data, var):
    print(pd.crosstab(index=data['scale'], columns=data[var]))
    print()
    print(pd.crosstab(index=data['scale'], columns=data[var], normalize='columns').round(3))

In [ ]:
df_raw.columns

Index(['plate_no', 'rolling_date', 'scale', 'spec_long', 'spec_country',
       'steel_kind', 'pt_thick', 'pt_width', 'pt_length', 'hsb', 'fur_no',
       'fur_input_row', 'fur_heat_temp', 'fur_heat_time', 'fur_soak_temp',
       'fur_soak_time', 'fur_total_time', 'fur_ex_temp', 'rolling_method',
       'rolling_temp', 'descaling_count', 'work_group'],
      dtype='object')

범주형 설명변수 : 'spec_long','spec_country','steel_kind','hsb', 'fur_no',
       'fur_input_row','rolling_method','work_group'

In [ ]:
fun_print_crosstab(df_raw, 'spec_country')

spec_country  공통  독일   미국   영국  일본  프랑스  한국
scale                                      
불량             3  13   44   75  97   29  49
양품            33  74  115  181  90  114  83

spec_country     공통     독일     미국     영국     일본    프랑스     한국
scale                                                        
불량            0.083  0.149  0.277  0.293  0.519  0.203  0.371
양품            0.917  0.851  0.723  0.707  0.481  0.797  0.629


규격 나라별로 불량률의 차이가 유의미하다.  
규격의 길이, 두께, 폭의 차이가 불량률에 영향을 주는 것인지 의심해 볼 수 있다.

In [ ]:
fun_print_crosstab(df_raw, 'steel_kind')

steel_kind    C    T
scale               
불량          290   20
양품          465  225

steel_kind      C      T
scale                   
불량          0.384  0.082
양품          0.616  0.918


강종별로 불량률의 차이가 유의미하다.  
강종은 불량 발생에 영향을 주는 것으로 보인다.

In [ ]:
fun_print_crosstab(df_raw, 'hsb')

hsb    미적용   적용
scale          
불량      47  263
양품       0  690

hsb    미적용     적용
scale            
불량     1.0  0.276
양품     0.0  0.724


hsb 미적용 후판은 모두 불량.  
hsb는 후판 불량에 무조건 영향을 준다고 가정하고 hsb를 설명변수에서 제거하고 분석 진행.

In [ ]:
fun_print_crosstab(df_raw, 'fur_no')

fur_no  1호기  2호기  3호기
scale                
불량      101   92  117
양품      232  233  225

fur_no    1호기    2호기    3호기
scale                      
불량      0.303  0.283  0.342
양품      0.697  0.717  0.658


호기별로 불량률의 차이가 커보이지는 않지만 정확한 판단을 위해 검정이 필요하다.  
카이제곱?

In [ ]:
fun_print_crosstab(df_raw, 'fur_input_row')

fur_input_row   1열   2열
scale                  
불량             165  145
양품             343  347

fur_input_row     1열     2열
scale                      
불량             0.325  0.295
양품             0.675  0.705


장입열별로 불량률의 차이가 커보이지 않지만, 정확한 판단을 위해 검정이 필요하다.

In [ ]:
fun_print_crosstab(df_raw, 'rolling_method')

rolling_method  CR(제어압연)  TMCP(온도제어)
scale                               
불량                   297          13
양품                   543         147

rolling_method  CR(제어압연)  TMCP(온도제어)
scale                               
불량                 0.354       0.081
양품                 0.646       0.919


제어압연의 불량률이 유의미하게 높다.  
압연방법은 불량 발생에 영향을 주는 것으로 보인다.

In [ ]:
fun_print_crosstab(df_raw, 'work_group')

work_group   1조   2조   3조   4조
scale                         
불량          109   85   66   50
양품          171  207  152  160

work_group     1조     2조     3조     4조
scale                                 
불량          0.389  0.291  0.303  0.238
양품          0.611  0.709  0.697  0.762


1조와 4조 사이의 불량률에 유의미한 차이가 있어보인다.  
정확한 판단을 위해 검정 필요

In [ ]:
fun_print_crosstab(df_raw, 'descaling_count')  #이산형

descaling_count  5    6   7    8   9    10
scale                                     
불량                6   28  20  122  31  103
양품                0  178   0  127   0  385

descaling_count   5      6    7     8    9      10
scale                                             
불량               1.0  0.136  1.0  0.49  1.0  0.211
양품               0.0  0.864  0.0  0.51  0.0  0.789


압연 descaling 횟수가 5,7,9회 일 때의 후판이 전부 불량이다.  
횟수가 홀수인 것이 공정 중 어떤 의미를 가지는지 알아볼 필요가 있다.